In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [25]:
# a = [1,2,3]
# b = [4,5,6]
# zipped_list = zip(a,b)
zipped_list=[(1, 4), (3, 5), (2, 6)]
zipped_list
unzipped_list=list(zip(*zipped_list))
unzipped_list
list(unzipped_list[0])
list(unzipped_list[1])

[(1, 4), (3, 5), (2, 6)]

[(1, 3, 2), (4, 5, 6)]

[1, 3, 2]

[4, 5, 6]

In [36]:
p=[(1, 4), (3, 5), (2, 6)]

p.sort()
p

p=list(zip(*p))
p

p=[list(p[0]),list(p[1])]
p

[(1, 4), (2, 6), (3, 5)]

[(1, 2, 3), (4, 6, 5)]

[[1, 2, 3], [4, 6, 5]]

In [2]:
import numpy as np
from toolbox import *
from skeleton import *
import pandas as pd
from gensim.models import FastText



[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
obj=np.arange(100*50000).reshape((50000,100))
np.save("obj.npy",obj)

In [15]:
X = np.load("obj.npy", mmap_mode='r')
# X = np.load("obj.npy")
X[1000, :]

memmap([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
        152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
        165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
        178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
        191, 192, 193, 194, 195, 196, 197, 198, 199])

In [16]:
import sys
local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 88
__builtins__ 88
_ih 272
_oh 248
_dh 80
In 272
Out 248
get_ipython 72
exit 64
quit 64
_ 128
__ 96
___ 128
_i 119
_ii 158
_iii 119
_i1 167
_i2 167
np 88
defaultdict 400
datetime 88
os 88
pickle 88
random 88
re 88
tweet_iterator 144
stopwords 2904
SnowballStemmer 1064
TfidfVectorizer 1064
sp 88
show_progress 144
mtxvec2array 144
count_nonzero_triangular 144
cleanStr 144
nltk 88
removeStopWords 144
stemmer 64
stemming 144
process_line 144
InvertedIdx 1064
skmeans 1064
transforms 88
plt 88
cm 88
itertools 88
string 88
unicodedata 88
word_tokenize 144
PCA 1064
pd 88
faiss 88
axes3d 88
punctuation_table 1192
stop_words 2904
euclidiana 144
coseno 144
plotClusters 144
plotPCA 144
preprocess 144
tokenize_sentences 144
cocurrency_matrix 144
plotDecisionBoundary 144
NearestCentroid 1064
kNN 1064
Clustering 1064
FastText 1064
_i3 78
_3 112
_i4 105
obj 158
_i5 108
_i6 108
_i7 105
_i8 94
X 144
_8 128
_i9 158
sys 88
local_